In [1]:
import keras
import keras.backend as K
from keras import initializers
import h5py
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

In [25]:
import numpy as np
import random

class G():
  
  def __init__(self, model_fixed, clean_dataset,
               num_overlay=6,
               clean_data_overlay_rate=0.5,
               input_data_overlay_rate=0.9,
               cls=1283,
               threshold = 0.1):
    # super(G, self).__init__()
    self.model_fixed = model_fixed
    self.clean_dataset = clean_dataset
    self.num_overlay = num_overlay
    self.clean_data_overlay_rate = clean_data_overlay_rate
    self.input_data_overlay_rate = input_data_overlay_rate
    self.cls = cls
    self.threshold = threshold
  
  def overlay(self, x, clean_img):
    overlay_img = x * self.input_data_overlay_rate + clean_img * self.clean_data_overlay_rate
    return np.clip(overlay_img, 0.0, 255.0)
  
  def predict(self, x):
    final_out = []
    for i in range(x.shape[0]):
      img_shape = x[i].shape
      overlay_res = np.zeros(self.num_overlay)
      for j in range(self.num_overlay):
        overlay_img = self.overlay(x[i],
                                   self.clean_dataset[random.randint(0, self.clean_dataset.shape[0]-1)])
        overlay_img = np.asarray(overlay_img).reshape((1,
                                                       img_shape[0],
                                                       img_shape[1],
                                                       img_shape[2]))
        overlay_res[j] = np.argmax(self.model_fixed.predict(overlay_img))
      if np.var(overlay_res) < self.threshold:
        # smaller variance on overlayed prediction => poisoned
        output_G = np.zeros(self.cls + 1)
        output_G[self.cls] = 1
      else:
        # bigger variance on overlayed prediction => not poisoned
        x_single = np.asarray(x[i]).reshape((1,
                                             img_shape[0],
                                             img_shape[1],
                                             img_shape[2]))
        output_G = np.append(self.model_fixed.predict(x_single), 0)
      final_out.append(output_G)
      if i%1000 == 0:
        print('Finished iter: ', i)
    return np.array(final_out)

In [26]:
# supposely "eval.py" for alternative G class


clean_data_filename = '/content/gdrive/MyDrive/ml_cyber_data/clean_test_data.h5'
clean_data_val_filename = '/content/gdrive/MyDrive/ml_cyber_data/clean_validation_data.h5'

poisoned_data_filename1 = '/content/gdrive/MyDrive/ml_cyber_data/multi/eyebrows_poisoned_data.h5'
poisoned_data_filename2 = '/content/gdrive/MyDrive/ml_cyber_data/multi/lipstick_poisoned_data.h5'
poisoned_data_filename3 = '/content/gdrive/MyDrive/ml_cyber_data/multi/sunglasses_poisoned_data.h5'



repaired_model_filename = '/content/gdrive/MyDrive/ml_cyber_data/sunglasses_bd_net.h5'


"""         THIS PATH FORMAT IS FOR .PY FILE
clean_data_filename = '/data/clean_test_data.h5'
poisoned_data_filename = '/data/sunglasses_poisoned_data.h5'

repaired_model_filename = 'models/repaired_model_30.h5'
"""
"""
Alternatively, use 
'models/repaired_model_2.h5', or
'models/repaired_model_4.h5', or
'models/repaired_model_10.h5'
for repaired_model_filename
to experiment with different repaired models based on their 
fractions of pruned channels
"""

cl_x_test, cl_y_test = data_loader(clean_data_filename)
bd_x_test1, bd_y_test1 = data_loader(poisoned_data_filename1)
bd_x_test2, bd_y_test2 = data_loader(poisoned_data_filename2)
bd_x_test3, bd_y_test3 = data_loader(poisoned_data_filename3)
clean_val, clean_val_y = data_loader(clean_data_val_filename)


repaired_model = keras.models.load_model(repaired_model_filename, compile=False)

goodnet = G(repaired_model, clean_val)



In [27]:
print('Eyebrows Poisoned Data')
cl_label_p = np.argmax(goodnet.predict(cl_x_test), axis=1)
clean_accuracy = np.mean(np.equal(cl_label_p, cl_y_test)) * 100
print('Clean Classification accuracy:', clean_accuracy)

bd_label_p = np.argmax(goodnet.predict(bd_x_test1), axis=1)
asr = np.mean(np.equal(bd_label_p, bd_y_test1)) * 100
print('Attack Success Rate:', asr)

print('Lipstick Poisoned Data')
print('Clean Classification accuracy:', clean_accuracy)

bd_label_p = np.argmax(goodnet.predict(bd_x_test2), axis=1)
asr = np.mean(np.equal(bd_label_p, bd_y_test2)) * 100
print('Attack Success Rate:', asr)

print('Sunglasses Poisoned Data')
print('Clean Classification accuracy:', clean_accuracy)

bd_label_p = np.argmax(goodnet.predict(bd_x_test3), axis=1)
asr = np.mean(np.equal(bd_label_p, bd_y_test3)) * 100
print('Attack Success Rate:', asr)

Eyebrows Poisoned Data
Finished iter:  0
Finished iter:  1000
Finished iter:  2000
Finished iter:  3000
Finished iter:  4000
Finished iter:  5000
Finished iter:  6000
Finished iter:  7000
Finished iter:  8000
Finished iter:  9000
Finished iter:  10000
Finished iter:  11000
Finished iter:  12000
Clean Classification accuracy: 92.58768511301638
Finished iter:  0
Finished iter:  1000
Finished iter:  2000
Finished iter:  3000
Finished iter:  4000
Finished iter:  5000
Finished iter:  6000
Finished iter:  7000
Finished iter:  8000
Finished iter:  9000
Finished iter:  10000
Attack Success Rate: 0.0779423226812159
Lipstick Poisoned Data
Clean Classification accuracy: 92.58768511301638
Finished iter:  0
Finished iter:  1000
Finished iter:  2000
Finished iter:  3000
Finished iter:  4000
Finished iter:  5000
Finished iter:  6000
Finished iter:  7000
Finished iter:  8000
Finished iter:  9000
Finished iter:  10000
Attack Success Rate: 0.0779423226812159
Sunglasses Poisoned Data
Clean Classification